# Project 1: SAT & ACT Analysis

The first markdown cell in a notebook is a great place to provide an overview of your entire project. You will likely want to at least state your

## Problem Statement

- To analyze the SAT and ACT Tests taken in 2017 and 2018.
- To analyze the aggregated scores, participation rates for each state in United States.
- Provide recommendations to the College Board to improve participation rates for future tests.

## Executive Summary

If you want to, it's great to use relative links to direct your audience to various sections of a notebook. **HERE'S A DEMONSTRATION WITH THE CURRENT SECTION HEADERS**:

### Contents:
- [2017 Data Import & Cleaning](#Data-Import-and-Cleaning)
- [2018 Data Import and Cleaning](#2018-Data-Import-and-Cleaning)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Data Visualization](#Visualize-the-data)
- [Descriptive and Inferential Statistics](#Descriptive-and-Inferential-Statistics)
- [Outside Research](#Outside-Research)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)

**If you combine your problem statement, executive summary, data dictionary, and conclusions/recommendations, you have an amazing README.md file that quickly aligns your audience to the contents of your project.** Don't forget to cite your data sources!

*All libraries used should be added here*

In [ ]:
# import libraries

# maths
import scipy.stats as stats
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

# visual
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# other
import re
import os

# html
from IPython.display import Image
from IPython.core.display import HTML

## 2017 Data Import and Cleaning

#### 1. Read In SAT & ACT  Data

Read in the `sat_2017.csv` and `act_2017.csv` files and assign them to appropriately named pandas dataframes.

In [ ]:
# import 2017 data

path = '../data/'

df_sat_2017 = pd.read_csv(path + 'sat_2017.csv')
df_act_2017 = pd.read_csv(path + 'act_2017.csv')

#### 2. Display Data

Print the first 10 rows of each dataframe to your jupyter notebook

In [ ]:
# verify df_sat_2017

df_sat_2017.head(n=10)

#df_sat_2017.tail(n=10)
#print(len(df_sat_2017))
#print(df_sat_2017)

In [ ]:
# verify df_act_2017

df_act_2017.head(n=10)

#df_act_2017.tail(n=10)
#print(len(df_act_2017))
#print(df_act_2017)

#### 3. Verbally Describe Data

Take your time looking through the data and thoroughly describe the data in the markdown cell below. 

[2017 SAT Data]

- The table shows the SAT scores for all 50 states in United States.
- It shows the average scores for 2 sections "Evidence-Based Reading" and Writing" and "Maths".
- The participation rate and total score are also listed.

[2017 ACT Data]

- The table shows the ACT scores for all 50 states in United States.
- The average scores for 4 sections are listed e.g. "Science" and "Composite".
- The participation rate and composite score (average) are listed.

#### 4a. Does the data look complete? 

- The data looks incomplete. 
- "Participation" lacks clarity and meaning. 
- Does it mean the participation rate of the entire population or high school students by state?
- "Number of Residents" or "Number of Students" column will be useful to add context for further EDA.

#### 4b. Are there any obvious issues with the observations?

- "National" is listed in df_act_2017['State'] but it is not a US State.
- This row should be removed as it shows the the aggregated scores for the different sections.

**What is the minimum *possible* value for each test/subtest? What is the maximum *possible* value?**

Consider comparing any questionable values to the sources of your data:
- [SAT](https://blog.collegevine.com/here-are-the-average-sat-scores-by-state/)
- [ACT](https://blog.prepscholar.com/act-scores-by-state-averages-highs-and-lows)

In [ ]:
%%html
<style>
table {float:left}
</style>

| SAT SCORING | Evidence-Based Reading | Maths | Total |
|---          |---                     |---    |---    |
|Max          |800                     |800    |1600   |     
|Min          |200                     |200    |400    |

| ACT SCORING | English| Maths | Reading | Science | Composite |
|-------------|--------|-------|---------|---------|-----------|
|Max          |36      |36     |36       |36       |36         |    
|Min          |1       |1      |1        |1        |1          |
</p>

#### 4c. Fix any errors you identified

**The data is available** so there's no need to guess or calculate anything. If you didn't find any errors, continue to the next step.

In [ ]:
# remove "National" row from df_act_2017

#print(len(df_act_2017))

mask = df_act_2017['State'] == 'National'
df_act_2017 = df_act_2017[~mask]

df_act_2017.head()

#df_act_2017.tail()
#print(len(df_act_2017))

#### 5. What are your data types? 
Display the data types of each feature. 

In [ ]:
print('df_sat_2017 (DataType):\n')
print(df_sat_2017.dtypes)
print('')
print('df_act_2017 (DataType):\n')
print(df_act_2017.dtypes)

What did you learn?
- Do any of them seem odd?  
- Which ones are not as they should be?  

| df_sat_2017                      | input type | correct type |
|----------------------------------|------------|--------------|
State                              |   object   | string       |
Participation                      |   object   | float64      |
Evidence-Based Reading and Writing |    int64   | int64        |
Math                               |    int64   | int64        |
Total                              |    int64   | int64        |

| df_act_2017 | input type | correct type |
|-------------|------------|--------------|
State         |   object   | string       |
Participation |   object   | float64      |
English       |  float64   | float64      |
Math          |  float64   | float64      |
Reading       |  float64   | float64      |
Science       |  float64   | float64      |
Composite     |   object   | float64      |

#### 6. Fix Incorrect Data Types
Based on what you discovered above, use appropriate methods to re-type incorrectly typed data.
- Define a function that will allow you to convert participation rates to an appropriate numeric type. Use `map` or `apply` to change these columns in each dataframe.

In [ ]:
def percent_str_to_float(x):  
    
    if isinstance(x,str) == True:    
        x = x.replace('%','')
        x = x.replace(')','')
        
    x = float(x)/100
        
    return x

In [ ]:
df_sat_2017['Participation'] = df_sat_2017['Participation'].apply(percent_str_to_float)
df_act_2017['Participation'] = df_act_2017['Participation'].apply(percent_str_to_float)

#df_sat_2017
#df_act_2017

- Fix any individual values preventing other columns from being the appropriate type.

In [ ]:
def remove_non_digit_char(x):   
    
    if isinstance(x,str) == True:
        x = re.sub(r'[a-zA-Z]','',x)    
        
    return x

In [ ]:
cols = df_sat_2017.columns
# skip "State" column
for col in cols[1:]:
    df_sat_2017[col] = df_sat_2017[col].apply(remove_non_digit_char)
    
cols = df_act_2017.columns
# skip "State" column
for col in cols[1:]:
    #print(col)
    df_act_2017[col] = df_act_2017[col].apply(remove_non_digit_char)

- Finish your data modifications by making sure the columns are now typed appropriately.

In [ ]:
# "State" column: object -> string
df_sat_2017["State"] = df_sat_2017["State"].astype(str)
df_act_2017["State"] = df_act_2017["State"].astype(str)

# "Composite" column: object -> float
df_act_2017["Composite"] = df_act_2017["Composite"].astype(float)

- Display the data types again to confirm they are correct.

In [ ]:
print('df_sat_2017 (DataType):\n')
print(df_sat_2017.dtypes)
print('')
print('df_act_2017 (DataType):\n')
print(df_act_2017.dtypes)

#### 7. Rename Columns
Change the names of the columns to more expressive names so that you can tell the difference the SAT columns and the ACT columns. Your solution should map all column names being changed at once (no repeated singular name-changes). **We will be combining these data with some of the data from 2018, and so you should name columns in an appropriate way**.

**Guidelines**:
- Column names should be all lowercase (you will thank yourself when you start pushing data to SQL later in the course)
- Column names should not contain spaces (underscores will suffice--this allows for using the `df.column_name` method to access columns in addition to `df['column_name']`.
- Column names should be unique and informative (the only feature that we actually share between dataframes is the state).

In [ ]:
def add_sat(col_name):
    
    if col_name == 'state':
        return col_name
        
    return 'sat_' + col_name 

In [ ]:
def add_act(col_name):
    
    if col_name == 'state':
        return col_name
        
    return 'act_' + col_name 

In [ ]:
# rename column names -> change to lowercase
df_sat_2017.columns = map(str.lower, df_sat_2017.columns)
df_act_2017.columns = map(str.lower, df_act_2017.columns)

In [ ]:
# rename columns with long names
df_sat_2017 = df_sat_2017.rename(columns={'evidence-based reading and writing':'read_write'})

# add sat_/act_ to column names
df_sat_2017.columns = map(add_sat, df_sat_2017.columns)
df_act_2017.columns = map(add_act, df_act_2017.columns)

In [ ]:
# fix error (found using histogram much below) in df_sat_2017 "Maryland" "Maths"

mask = df_sat_2017["state"] == "Maryland"
row = df_sat_2017.loc[mask]

print("before:")
print(row)

total = row["sat_total"]
read_write = row["sat_read_write"]
math = total - read_write

df_sat_2017.loc[mask,"sat_math"] = math

row = df_sat_2017.loc[mask]
print("after:")
print(row)

In [ ]:
# fix error (found using histogram much below) in df_act_2017 "Maryland" "Science"

mask = df_act_2017["state"] == "Maryland"
row = df_act_2017.loc[mask]

print("before:")
print(row)

english = row["act_english"]
math = row["act_math"]
reading = row["act_reading"]
composite = row["act_composite"]

science = (composite*4) - english - math - reading
print(science)


df_act_2017.loc[mask,"act_science"] = science

row = df_act_2017.loc[mask]
print("after:")
print(row)

#### 8. Create a data dictionary

Now that we've fixed our data, and given it appropriate names, let's create a [data dictionary](http://library.ucmerced.edu/node/10249). 

A data dictionary provides a quick overview of features/variables/columns, alongside data types and descriptions. The more descriptive you can be, the more useful this document is.

Example of a Fictional Data Dictionary Entry: 

|Feature|Type|Dataset|Description|
|---|---|---|---|
|**county_pop**|*integer*|2010 census|The population of the county (units in thousands, where 2.5 represents 2500 people).| 
|**per_poverty**|*float*|2010 census|The percent of the county over the age of 18 living below the 200% of official US poverty rate (units percent to two decimal places 98.10 means 98.1%)|

[Here's a quick link to a short guide for formatting markdown in Jupyter notebooks](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html).

Provided is the skeleton for formatting a markdown table, with columns headers that will help you create a data dictionary to quickly summarize your data, as well as some examples. **This would be a great thing to copy and paste into your custom README for this project.**

|Feature|Type|Dataset|Description|
|---|---|---|---|
|**state**|*string*| SAT / ACT | A region in USA |
|**year**|*int*| SAT / ACT | States year the test was taken |
|**sat_participation**|*int*| SAT | Percentage of high students who took the SAT Test |
|**sat_read_write**|*int*| SAT | Composite score for "Evidence-Based Reading and Writing"  |
|**sat_math**|*int*| SAT | Composite score for Math |
|**sat_total**|*int*| SAT | Total score for SAT Test |
|**act_participation**|*int*| ACT | Percentage of high students who took the ACT Test |
|**act_english**|*float*| ACT | Composite score for English |
|**act_math**|*float*| ACT | Composite score for Math |
|**act_reading**|*float*| ACT | Composite score for Reading |
|**act_science**|*float*| ACT | Composite score for Science |
|**act_composite**|*float*| ACT | Composite score for ACT Test |

#### 9. Drop unnecessary rows

One of our dataframes contains an extra row. Identify and remove this from the dataframe.

In [ ]:
# no action -> extra row already dropped in earlier cell

#### 10. Merge Dataframes

Join the 2017 ACT and SAT dataframes using the state in each dataframe as the key. Assign this to a new variable.

In [ ]:
# merge df_sat_2017 with df_act_2017 -> df_2017
df_2017 = pd.merge(df_sat_2017,df_act_2017,on='state')

# add new column year
df_2017.insert(1, 'year', 2017)

#### 11. Save your cleaned, merged dataframe

Use a relative path to save out your data as `combined_2017.csv`.

In [ ]:
# output combined 2017 data

path = '../output/'

if not os.path.exists(path):
    os.makedirs(path)

df_2017.to_csv(path + 'combined_2017.csv')

## 2018 Data Import and Cleaning

Links to the 2018 ACT and SAT data are provided in the README. These data live in PDFs, and so you'll get to enjoy practicing some *manual* data collection. Save these data as a CSV in your `data` directory, and import, explore, and clean these data in the same way you did above. **Make sure you comment on your steps so it is clear *why* you are doing each process**.

In [ ]:
# import 2018 data

path = '../data/'

df_sat_2018 = pd.read_csv(path + 'sat_2018.csv')
df_act_2018 = pd.read_csv(path + 'act_2018.csv')

#df_sat_2018
#df_act_2018

In [ ]:
def remove_word_avg_score(col_name):
    col_name = col_name.replace('Average ','')
    col_name = col_name.replace(' Score','')
    return col_name

In [ ]:
def change_to_nan(value):
    if value == 'Not given' or value == None:
        print(value)
        return np.nan
    else:
        return value

In [ ]:
# clean data: df_sat_2018 and df_act_2018

# split column with state and participation info
df_split = df_sat_2018["State"].str.split(" \(", n = 1, expand = True) 

# add back state and participation columns
df_sat_2018["State"] = df_split[0]
df_sat_2018["Participation"] = df_split[1]

In [ ]:
# remove "National" row
mask = df_act_2018['State'] == 'National'
df_act_2018 = df_act_2018[~mask]

# rename columns directly
df_sat_2018 = df_sat_2018.rename(columns={'EBRW':'read_write'})
df_act_2018 = df_act_2018.rename(columns={'Percentage of Students Tested':'Participation'})

In [ ]:
# change values in participation column
df_sat_2018["Participation"] = df_sat_2018['Participation'].apply(change_to_nan)
df_act_2018["Participation"] = df_act_2018['Participation'].apply(change_to_nan)
df_sat_2018['Participation'] = df_sat_2018['Participation'].apply(percent_str_to_float)
df_act_2018['Participation'] = df_act_2018['Participation'].apply(percent_str_to_float)

# drop irrelevant column
df_sat_2018 = df_sat_2018.drop(columns=["# of Test Takers"])

In [ ]:
# remove irrelevant words in column names
df_act_2018.columns = map(remove_word_avg_score, df_act_2018.columns)
df_act_2018.columns = map(remove_word_avg_score, df_act_2018.columns)

# rename column names -> change to lowercase
df_sat_2018.columns = map(str.lower, df_sat_2018.columns)
df_act_2018.columns = map(str.lower, df_act_2018.columns)

# add sat_/act_ to column names
df_sat_2018.columns = map(add_sat, df_sat_2018.columns)
df_act_2018.columns = map(add_act, df_act_2018.columns)

In [ ]:
# "object -> string
#df_sat_2018["state"] = df_sat_2018["state"].astype(str)
#df_act_2018["state"] = df_act_2018["state"].astype(str)

# object -> float
#df_sat_2018["participation"] = df_sat_2018["participation"].astype(float)
#df_act_2018["participation"] = df_act_2018["participation"].astype(float)

In [ ]:
# merge and create df_2018
df_2018 = pd.merge(df_sat_2018,df_act_2018,on='state')

# add year column
df_2018['year'] = 2018

#### Combine your 2017 and 2018 data into a single dataframe
Joining on state names should work, assuming you formatted all your state names identically. Make sure none of your columns (other than state) have identical names. Do yourself a favor and decide if you're encoding participation rates as floats or integers and standardize this across your datasets.

Save the contents of this merged dataframe as `final.csv`.

**Use this combined dataframe for the remainder of the project**.

In [ ]:
# combine df_2017 and df_2018
df_final = df_2017.append(df_2018,sort=False)

In [ ]:
# convert percentage e.g. 0.12 to 12

df_final['sat_participation'] = df_final['sat_participation'] * 100
df_final['act_participation'] = df_final['act_participation'] * 100

In [ ]:
# output combined data

path = '../output/'

if not os.path.exists(path):
    os.makedirs(path)

df_final.to_csv(path + 'final.csv')

## Exploratory Data Analysis


### Summary Statistics
Transpose the output of pandas `describe` method to create a quick overview of each numeric feature.

In [ ]:
# CHECK!!! act_participation is missing

summary = df_final.describe()
summary.transpose()

#### Manually calculate standard deviation

$$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^n(x_i - \mu)^2}$$

- Write a function to calculate standard deviation using the formula above

In [ ]:
def compute_stdev(col):
    
    if is_numeric_dtype(col) == False:
        return
    
    n = len(col)
    mean = np.mean(col)
    total = 0
    
    for x in col:
        total += (x-mean)**2
        
    variance = total / n
    stdev = np.sqrt(variance)
    
    print(n,mean,stdev)    
    
    #return round(stdev,2)
    return stdev

- Use a **dictionary comprehension** to apply your standard deviation function to each numeric column in the dataframe.  **No loops**  
- Assign the output to variable `sd` as a dictionary where: 
    - Each column name is now a key 
    - That standard deviation of the column is the value 
     
*Example Output :* `{'ACT_Math': 120, 'ACT_Reading': 120, ...}`

In [ ]:
sd = {col:compute_stdev(df_final[col]) for col in df_final if col != 'state'}

print(sd)

Do your manually calculated standard deviations match up with the output from pandas `describe`? What about numpy's `std` method?

Answer

#### Investigate trends in the data
Using sorting and/or masking (along with the `.head` method to not print our entire dataframe), consider the following questions:

- Which states have the highest and lowest participation rates for the:
    - 2017 SAT?
    - 2018 SAT? 
    - 2017 ACT? 
    - 2018 ACT?
- Which states have the highest and lowest mean total/composite scores for the:
    - 2017 SAT?
    - 2018 SAT?
    - 2017 ACT?
    - 2018 ACT?
- Do any states with 100% participation on a given test have a rate change year-to-year?
- Do any states show have >50% participation on *both* tests either year?

Based on what you've just observed, have you identified any states that you're especially interested in? **Make a note of these and state *why* you think they're interesting**.

**You should comment on your findings at each step in a markdown cell below your code block**. Make sure you include at least one example of sorting your dataframe by a column, and one example of using boolean filtering (i.e., masking) to select a subset of the dataframe.

[Highest Participation Rate]

- 2017 SAT: District of Columbia, Michigan, Connecticut, Delaware, New Hampshire
- 2018 SAT: Colorado, Connecticut, Delaware, Michigan, Idaho
- 2017 ACT: Alabama, Kentucky, Wisconsin, Utah, Tennessee
- 2018 ACT: Alabama, Kentucky, Wisconsin, Utah, Tennessee

[Lowest Participation Rate]

- 2017 SAT: North Dakota, Mississippi, Iowa, Missouri, Utah
- 2018 SAT: North Dakota, Wyoming, South Dakota, Nebraska, Wisconsin
- 2017 ACT: Maine, New Hampshire, Delaware, Rhode Island, Pennsylvania
- 2018 ACT: Maine, Rhode Island, New Hampshire, Delaware, North Carolina

[Highest Total/Composite Rate]

- 2017 SAT: Minnesota, Wisconsin, Iowa, Missouri, Kansas
- 2018 SAT: Minnesota, Wisconsin, North Dakota, Iowa, Kansas
- 2017 ACT: New Hampshire, Massachusetts, Connecticut, Maine, District of Columbia
- 2018 ACT: Connecticut, Massachusetts, New Hampshire, New York, Michigan

[Lowest Total/Composite Rate]

- 2017 SAT: District of Columbia, Delaware, Idaho, Michigan, Maine
- 2018 SAT: District of Columbia, Delaware, West Virginia, Idaho, Michigan
- 2017 ACT: Nevada, Mississippi, South Carolina, Hawaii, North Carolina
- 2018 ACT: Nevada, South Carolina, Mississippi, Hawaii, Alabama

In [ ]:
# create masks
mask_2017 = df_final['year'] == 2017
mask_2018 = df_final['year'] == 2018

In [ ]:
# SAT 2017 highest participation rates
df_sat_2017_high_participation = df_final[mask_2017].sort_values('sat_participation',ascending=False)
df_sat_2017_high_participation.head()

In [ ]:
# SAT 2018 highest participation rates
df_sat_2018_high_participation = df_final[mask_2018].sort_values('sat_participation',ascending=False)
df_sat_2018_high_participation.head()

In [ ]:
# ACT 2017 highest participation rates
df_act_2017_high_participation = df_final[mask_2017].sort_values('act_participation',ascending=False)
df_act_2017_high_participation.head()

In [ ]:
# ACT 2018 highest participation rates
df_act_2018_high_participation = df_final[mask_2018].sort_values('act_participation',ascending=False)
df_act_2018_high_participation.head()

In [ ]:
# SAT 2017 lowest participation rates
df_sat_2017_low_participation = df_final[mask_2017].sort_values('sat_participation',ascending=True)
df_sat_2017_low_participation.head()

In [ ]:
# SAT 2018 lowest participation rates
df_sat_2018_low_participation = df_final[mask_2018].sort_values('sat_participation',ascending=True)
df_sat_2018_low_participation.head()

In [ ]:
# ACT 2017 lowest participation rates
df_act_2017_low_participation = df_final[mask_2017].sort_values('act_participation',ascending=True)
df_act_2017_low_participation.head()

In [ ]:
# ACT 2018 lowest participation rates
df_act_2018_low_participation = df_final[mask_2018].sort_values('act_participation',ascending=True)
df_act_2018_low_participation.head()

In [ ]:
# SAT 2017 highest total score
df_sat_2017_high_total = df_final[mask_2017].sort_values('sat_total',ascending=False)
df_sat_2017_high_total.head()

In [ ]:
# SAT 2018 highest total score
df_sat_2018_high_total = df_final[mask_2018].sort_values('sat_total',ascending=False)
df_sat_2018_high_total.head()

In [ ]:
# ACT 2017 highest composite score
df_act_2017_high_composite = df_final[mask_2017].sort_values('act_composite',ascending=False)
df_act_2017_high_composite.head()

In [ ]:
# ACT 2018 highest composite score
df_act_2018_high_composite = df_final[mask_2018].sort_values('act_composite',ascending=False)
df_act_2018_high_composite.head()

In [ ]:
# SAT 2017 lowest total score
df_sat_2017_low_total = df_final[mask_2017].sort_values('sat_total',ascending=True)
df_sat_2017_low_total.head()

In [ ]:
# SAT 2018 lowest total score
df_sat_2018_low_total = df_final[mask_2018].sort_values('sat_total',ascending=True)
df_sat_2018_low_total.head()

In [ ]:
# ACT 2017 lowest composite score
df_act_2017_low_composite = df_final[mask_2017].sort_values('act_composite',ascending=True)
df_act_2017_low_composite.head()

In [ ]:
# ACT 2018 lowest composite score
df_act_2018_low_composite = df_final[mask_2018].sort_values('act_composite',ascending=True)
df_act_2018_low_composite.head()

## Visualize the data

There's not a magic bullet recommendation for the right number of plots to understand a given dataset, but visualizing your data is *always* a good idea. Not only does it allow you to quickly convey your findings (even if you have a non-technical audience), it will often reveal trends in your data that escaped you when you were looking only at numbers.

Some recommendations on plotting:
- Plots have titles
- Plots have axis labels
- Plots have appropriate tick labels
- All text is legible in a plot
- Plots demonstrate meaningful and valid relationships
- Plots are interpreted to aid understanding

There is such a thing as too many plots, and there are a *lot* of bad plots. You might make some! (But hopefully not with the guided prompts below).

#### Use Seaborn's heatmap with pandas `.corr()` to visualize correlations between all numeric features

Heatmaps are generally not appropriate for presentations, and should often be excluded from reports as they can be visually overwhelming. **However**, they can be extremely useful in identify relationships of potential interest (as well as identifying potential collinearity before modeling).

*example*:
```python
sns.heatmap(df.corr())
```

Please take time to format your output, adding a title. Look through some of the additional arguments and options. (Axis labels aren't really necessary, as long as the title is informative).

In [ ]:
# df_final heatmap
fig, ax = plt.subplots(figsize=(18, 8))
sns.heatmap(df_final.corr(),annot=True)

In [ ]:
# df_final pairplots
sns.pairplot(df_final)

#### Define a custom function to subplot histograms

We have data for two tests for two years. We only have composite (and not subtest scores) for the 2018 ACT. We should write a function that will take the names of 2+ columns and subplot histograms. While you can use pandas plotting or Seaborn here, matplotlib gives you greater control over all aspects of your plots.

[Helpful Link for Plotting Multiple Figures](https://matplotlib.org/users/pyplot_tutorial.html#working-with-multiple-figures-and-axes)

Here's some starter code:

#### Plot and interpret histograms 
For each of the following:
- Participation rates for SAT & ACT
- Math scores for SAT & ACT
- Reading/verbal scores for SAT & ACT

In [ ]:
# original

#def subplot_histograms(df, list_of_columns, list_of_titles, list_of_xlabels):
    #nrows = int(np.ceil(len(list_of_columns)/2) # Makes sure you have enough rows
    #fig, ax = plt.subplots(nrows=nrows, ncols=2) # You'll want to specify your figsize
    #ax = ax.ravel() # Ravel turns a matrix into a vector, which is easier to iterate
    #for i, column in enumerate(list_of_columns): # Gives us an index value to get into all our lists
        #ax[i].hist(dataframe[column]) # feel free to add more settings
        # Set titles, labels, etc here for each subplot

In [ ]:
# modified

def subplot_histograms(df,rows,cols,titles,xlabels,ylabels,ylims,bins):
    
    # Makes sure you have enough rows
    nrows = len(rows)   
    ncols = len(rows)    
    #print(nrows,ncols)
                
    # You'll want to specify your figsize            
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols,figsize=(16,12))
                
    # Ravel turns a matrix into a vector, which is easier to iterate            
    ax = ax.ravel()
    index = 0
                
    # Gives us an index value to get into all our lists            
    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
                
            #print(i,j)
            
            # feel free to add more settings        
            ax[index].hist(df[row][col],bins=bins)

            # Set titles, labels, etc here for each subplot
            ax[index].set_title(titles[index]) 
            ax[index].set_xlabel(xlabels)
            ax[index].set_ylabel(ylabels)
            ax[index].set_ylim(ylims[0],ylims[1])
            
            index += 1           

In [ ]:
# Participation Rates for SAT & ACT

rows = [mask_2018,mask_2017]
cols = ['sat_participation','act_participation']
titles = ['SAT 2018','ACT 2018','SAT 2017','ACT 2017']
xlabels = 'Percentage'
ylabels = 'No of States'
ylims = [0,25]
bins = 10

print("Participation Rates")
subplot_histograms(df_final,rows,cols,titles,xlabels,ylabels,ylims,bins)

In [ ]:
# Math scores for SAT & ACT

# Participation Rates for SAT & ACT

rows = [mask_2018,mask_2017]
cols = ['sat_math','act_math']
titles = ['SAT 2018','ACT 2018','SAT 2017','ACT 2017']
xlabels = 'Scores'
ylabels = 'Percentage'
ylims = [0,10]
bins = 15

print("Math Scores")
subplot_histograms(df_final,rows,cols,titles,xlabels,ylabels,ylims,bins)

In [ ]:
# create new column: average(act_english,act_reading) = act_verbal
df_final["act_verbal"] = (df_final["act_english"] + df_final["act_reading"]) / 2

In [ ]:
# Reading/verbal scores for SAT & ACT

rows = [mask_2018,mask_2017]
cols = ['sat_read_write','act_verbal']
titles = ['SAT 2018','ACT 2018','SAT 2017','ACT 2017']
xlabels = 'Scores'
ylabels = 'No of States'
ylims = [0,10]
bins = 15

print("Reading/Verbal Scores")
subplot_histograms(df_final,rows,cols,titles,xlabels,ylabels,ylims,bins)

#### Plot and interpret scatter plots

For each of the following:
- SAT vs. ACT math scores for 2017
- SAT vs. ACT verbal/reading scores for 2017
- SAT vs. ACT total/composite scores for 2017
- Total scores for SAT 2017 vs. 2018
- Composite scores for ACT 2017 vs. 2018

Plot the two variables against each other using matplotlib or Seaborn

Your plots should show:
- Two clearly labeled axes
- A proper title
- Using colors and symbols that are clear and unmistakable

**Feel free to write a custom function, and subplot if you'd like.** Functions save both time and space.


In [ ]:
df_final.head()

In [ ]:
def create_scatterplot(title,x,y,xlabel=None,ylabel=None,regplot=False):
    
    if xlabel == None:
        xlabel = x.name
    if ylabel == None:
        ylabel = y.name   
        
    if regplot:
        sns.regplot(x=x,y=y)
    else:
        sns.scatterplot(x=x,y=y)
                    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

In [ ]:
df_2017 = df_final[mask_2017]
df_2018 = df_final[mask_2018]

In [ ]:
#SAT vs. ACT math scores for 2017

x = df_2017['act_math']
y = df_2017['sat_math']
title = ("SAT vs. ACT math scores for 2017")

create_scatterplot(title,x,y)

In [ ]:
#SAT vs. ACT verbal/reading scores for 2017

x = df_2017['act_verbal']
y = df_2017['sat_read_write']
title = ("SAT vs. ACT verbal/reading scores for 2017")
         
create_scatterplot(title,x,y)

In [ ]:
#SAT vs. ACT total/composite scores for 2017

x = df_2017['act_composite']
y = df_2017['sat_total']
title = ("SAT vs. ACT total/composite scores for 20177")
         
create_scatterplot(title,x,y)

In [ ]:
#Total scores for SAT 2017 vs. 2018

x = df_2018['sat_total']
y = df_2017['sat_total']
title = ("Total scores for SAT 2017 vs. 2018")
         
create_scatterplot(title,x,y,2018,2017)

In [ ]:
#Composite scores for ACT 2017 vs. 2018

x = df_2018['act_composite']
y = df_2017['act_composite']
title = ("Composite scores for ACT 2017 vs. 2018")
         
create_scatterplot(title,x,y,2018,2017)

#### Plot and interpret boxplots

For each numeric variable in the dataframe create a boxplot using Seaborn. Boxplots demonstrate central tendency and spread in variables. In a certain sense, these are somewhat redundant with histograms, but you may be better able to identify clear outliers or differences in IQR, etc.

Multiple values can be plotted to a single boxplot as long as they are of the same relative scale (meaning they have similar min/max values).

Each boxplot should:
- Only include variables of a similar scale
- Have clear labels for each variable
- Have appropriate titles and labels

In [ ]:
def add_2017(col_name):
    return col_name + '_2017'

def add_2018(col_name):
    return col_name + '_2018'

In [ ]:
def create_boxplot(data,xlabel):
    
    g = sns.boxplot(data=data)
    #g.set(xlabel=xlabel)
    g.set(xticklabels=xlabel)

In [ ]:
# get individual scores for SAT

cols = ['sat_read_write','sat_math']

df_sat_2017 = df_2017[cols]
df_sat_2018 = df_2018[cols]

df_sat_2017.columns = map(add_2017, df_sat_2017.columns)
df_sat_2018.columns = map(add_2018, df_sat_2018.columns)

# combine 2017 and 2018 scores
df_sat = pd.concat([df_sat_2017, df_sat_2018], axis = 1)

In [ ]:
# reorder columns
new_order = ['sat_read_write_2017','sat_read_write_2018','sat_math_2017','sat_math_2018']
df_sat = df_sat.reindex(columns=new_order)

xlabels = ['Verbal 2017','Verbal 2018','Math 2017','Math 2018']
create_boxplot(df_sat,xlabels)

In [ ]:
# get total scores for SAT

cols = ['sat_total']

df_sat_2017 = df_2017[cols]
df_sat_2018 = df_2018[cols]

df_sat_2017.columns = map(add_2017, df_sat_2017.columns)
df_sat_2018.columns = map(add_2018, df_sat_2018.columns)

# combine 2017 and 2018 scores
df_sat = pd.concat([df_sat_2017, df_sat_2018], axis = 1)

In [ ]:
xlabels = ['Total 2017','Total 2018']
create_boxplot(df_sat,xlabels)

In [ ]:
# get scores for ACT 2017

cols = ['act_english','act_math','act_reading','act_science','act_composite']
df_act_2017 = df_2017[cols]

xlabels = ['English','Math','Reading','Science','Composite']
create_boxplot(df_act_2017,xlabels)

In [ ]:
# get scores for ACT 2018

cols = ['act_english','act_math','act_reading','act_science','act_composite']
df_act_2018 = df_2018[cols]

xlabels = ['English','Math','Reading','Science','Composite']
create_boxplot(df_act_2018,xlabels)

#### Feel free to do additional plots below
*(do research and choose your own chart types & variables)*

Are there any additional trends or relationships you haven't explored? Was there something interesting you saw that you'd like to dive further into? It's likely that there are a few more plots you might want to generate to support your narrative and recommendations that you are building toward. **As always, make sure you're interpreting your plots as you go**.

In [ ]:
#SAT vs. ACT math participation for 2017

x = df_2017['act_participation']
y = df_2017['sat_participation']
title = ("SAT vs. ACT math participation for 2017")
         
create_scatterplot(title,x,y,regplot=True)

In [ ]:
#SAT vs. ACT math participation for 2018

x = df_2018['act_participation']
y = df_2018['sat_participation']
title = ("SAT vs. ACT math participation for 2018")
         
create_scatterplot(title,x,y,regplot=True)

#### (Optional): Using Tableau, create a choropleth map for each variable using a map of the US. 

Save this plot as an image file in an images directory, provide a relative path, and insert the image into notebook in markdown.

In [ ]:
# Done using Excel

path = '../images/'
Image(filename = path + "map.png")

## Descriptive and Inferential Statistics

#### Summarizing Distributions

Above, we used pandas `describe` to provide quick summary statistics of our numeric columns. We also demonstrated many visual relationships.

As data scientists, having a complete understanding of data is imperative prior to modeling.

While we will continue to build our analytic tools, we know that measures of *central tendency*, *spread*, and *shape/skewness* provide a quick summary of distributions.

For each variable in your data, summarize the underlying distributions (in words & statistics)
 - Be thorough in your verbal description of these distributions.
 - Be sure to back up these summaries with statistics.

| variable | distribution | remarks |
|---|---|---|
| sat_participation | bimodal | 2 peaks at min and max |
| sat_read_write | bimodal | |
| sat_math | bimodal | |
| sat_total | bimodal | |
| act_participation | bimodal | 2 peaks at min and max |
| act_english | bimodal | |
| act_math | bimodal | |   
| act_reading | bimodal | |
| act_science | bimodal | |
| act_composite | bimodal | |
| act_verbal | bimodal | |

Note: Bimodal is a type of Normal Distribution i.e. 2 Normal Distributions with different Means.

In [ ]:
def histogram_matrix(df,nrows=2,ncols=2,bins=15):
    
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20,8))    
    ax = ax.ravel()
    
    index = 0
    
    color = 'r'
    
    for i, col in enumerate(df):
        
        if 'act_' in col:
            color = 'r'
        if 'sat_' in col:
            color = 'b'        
        
        #ax[index].hist(df[col], bins=bins)
        sns.distplot(df[col], hist=True, bins=bins, ax=ax[index],color=color)

        ax[index].set_title(col) 
        ax[index].set_xlabel('')            

        index += 1    

In [ ]:
cols = ['state','year','act_verbal']
df_final_2 = df_final.drop(cols,axis=1)
histogram_matrix(df_final_2,2,5,15)

#### We generally assuming that data we sample from a population will be normally distributed. Do we observe this trend?

- Yes to certain extend.
- Data shows bimodal distributions.

Does This Assumption Hold for:
    - Math
    - Reading
    - Rates
Explain your answers for each distribution and how you think this will affect estimates made from these data.

- Maths: Yes. Has 2 peaks. Peak #1 (with lower mean) has lower variance compared to Peak #2 (with higher mean).
- Reading. Same observation as Maths.
- Rates. Yes. Has 2 peaks at min and max values.

#### Estimate Limits of Data

Suppose we only seek to understand the relationship between SAT and ACT participation rates in 2017. 

##### Does it make sense to conduct statistical inference given these data specifically? 

Why or why not?

*(think about granularity, aggregation, the relationships between populations size & rates...consider the actually populations these data describe in answering this question)*

- No
- Population size of Class of 2017 and 2018 are not given for each state.
- Without population size it may not be possible to compare participation rate, scores between states, year or tests (SAT or ACT).

##### Is it appropriate to compare *these* specific SAT and ACT math scores? 

Why or why not?

[ACT vs SAT: 11 Key Differences to Help You Pick the Right Test] (https://blog.prepscholar.com/act-vs-sat)

- Time given to complete each question is different.
- SAT has a non-Calculator subsection. Not applicable for ACT.
- Different math topics are tested e.g. more focus in geometry for ACT and matrics tested in SAT.

Conclusion: Not appropriate to compare the SAT and ACT math scores.

#### Statistical Evaluation of Distributions 

**If you feel it's appropriate**, using methods we discussed in class, run hypothesis tests to compare variables of interest in our dataset. 

In [ ]:
# H0: The difference of SAT participation rate between 2017 and 2018 is zero.
# H1: The difference of SAT participation rate between 2017 and 2018 is not zero.
# alpha = 0.05

sat_part_2017 = list(df_final[mask_2017]['sat_participation'])
sat_part_2018 = list(df_final[mask_2018]['sat_participation'])

print(np.mean(sat_part_2017))
print(np.mean(sat_part_2018))

In [ ]:
t_stat, p_value = stats.ttest_ind(sat_part_2017, sat_part_2018)
print(t_stat, p_value)

In [ ]:
# p_value > alpha
# 0.34967 > 0.05

# conclusion:
# insufficient evidence to reject H0 -> The difference of SAT participation rate between 2017 and 2018 is zero.

In [ ]:
# Generate points on the x axis between -4 and 4:
xpoints = np.linspace(-4, 4, 500)

# Use `stats.t.pdf` to get values on the probability density function for the t-distribution.
# The second argument is the degrees of freedom: n1 + n2 - 2.
ypoints = stats.t.pdf(xpoints, (50+50-2), 0, 1)

In [ ]:
# Initialize a `matplotlib` "figure."
fig = plt.figure(figsize=(8,5))

# Get the current "axis" out of the figure.
ax = fig.gca()

# Plot the lines using `matplotlib`'s plot function:
ax.plot(xpoints, ypoints, linewidth=3, color='darkred')

# Plot a vertical line for our measured difference in rates' t-statistic.
ax.axvline(t_stat, color='black', linestyle='--', lw=5)
ax.axvline(-t_stat, color='black', linestyle='--', lw=5)

In [ ]:
#n_2017 = len(sat_part_2017)
#n_2018 = len(sat_part_2018)
#lower_tail = stats.t.cdf(-abs(t_stat), n_2017+n_2018-2)
#upper_tail = 1. - stats.t.cdf(abs(t_stat), n_2017+n_2018-2)
#p_value_man = lower_tail+upper_tail
#print(p_value)
#print(p_value_man)

## Outside Research

Based upon your observations, choose **three** states that demonstrate interesting trends in their SAT and/or ACT participation rates. Spend some time doing outside research on state policies that might influence these rates, and summarize your findings below. **Feel free to go back and create new plots that highlight these states of interest**. If you bring in any outside tables or charts, make sure you are explicit about having borrowed them. If you quote any text, make sure that it renders as being quoted. (Make sure that you cite your sources -- check with you local instructor for citation preferences).

In [ ]:
# find states with high participation rates for both tests

df_final["total_participation"] = df_final["sat_participation"] + df_final["act_participation"]
df_final.sort_values(by="total_participation",ascending=False).head(10)

In [ ]:
# states with low participation rates for both tests
df_final.sort_values(by="total_participation",ascending=True).head(n=10)

In [ ]:
# 1st state: Florida (high participation rates for both tests)

mask_florida = df_final['state'] == 'Florida'
df_florida = df_final[mask_florida]
df_florida

[Which States Require Students to Take the SAT or ACT?] (https://www.edweek.org/ew/section/multimedia/states-require-students-take-sat-or-act.html)

- Students in Florida must pass exit exams or achieve specified scores on ACT or SAT.

[SAT, ACT: Florida students lag behind national averages] (https://www.orlandosentinel.com/news/education/os-ne-act-sat-florida-scores-20181024-story.html)

- There is a 14% participation rate increase for SAT test in Florida between 2017 and 2018.
- This is because of free tests sessions provided by the schools.

In [ ]:
# 2nd state: Colorado (biggest delta for participation rates for SAT/ACT 2017/2018)

mask_colorado = df_final['state'] == 'Colorado'
df_colorado = df_final[mask_colorado]
df_colorado

[Which States Require Students to Take the SAT or ACT?] (https://www.edweek.org/ew/section/multimedia/states-require-students-take-sat-or-act.html)

- SAT/PSAT are required tests for high school students in Colorado.

[Colorado juniors face new, revamped college exam in SAT after state dumps rival ACT] (https://www.denverpost.com/2017/03/06/colorado-juniors-sat-college-exam/)

- The Colorado state decided to switch from ACT to SAT in 2015.
- The SAT test was better aligned with the high school Colorado Academic Standards.
- The College Board (which administrates the SAT test) also provides more education resources for students.

In [ ]:
# 3rd state: Iowa (low participations rates overall for both tests)

mask_iowa = df_final['state'] == 'Iowa'
df_iowa = df_final[mask_iowa]
df_iowa

[Which States Require Students to Take the SAT or ACT?] (https://www.edweek.org/ew/section/multimedia/states-require-students-take-sat-or-act.html)

- Both SAT and ACT are not required for Iowa.

[The History Of ACT Test] (https://www.testpreptoolkit.com/act-test/the-history-of-act-test)

- ACT was created by a University of Iowa professor (Everett Franklin Lindquist) in 1959.
- Influence students to take ACT instead of SAT?

## Conclusions and Recommendations

Based on your exploration of the data, what are you key takeaways and recommendations? Choose one state with a lower participation rate and provide a suggestion for how the College Board might increase participation amongst graduating seniors in this state. Are there additional data you desire that would better inform your investigations?

[Recommendations for Iowa]

1. Provide free test sessions for SAT tests.
2. Iowa Education Board should enforce either SAT or ACT tests for college admissions.

[Additional Data For Further Investigations]

1. College admission requirements for all colleges and universities in Iowa.
2. SAT and ACT test trends for 5 to 10 years.
3. Education demographic in Iowa (e.g. figures for high-school/college/university graduates).